# **Introduction**
Adaptive Retrieval-Augmented Generation (Adaptive-RAG) is an advanced extension of RAG that dynamically customizes retrieval strategies based on the query type, domain, and context sensitivity. Unlike traditional RAG models that use fixed retrieval techniques, Adaptive-RAG optimizes retrieval by switching between different retrieval strategies or adjusting retrieval parameters in real-time.

This approach improves retrieval efficiency, enhances relevance, and optimizes computational costs, making it ideal for multi-modal, multi-domain, and context-sensitive applications.

# **Key Concepts of Adaptive-RAG**
Adaptive-RAG is built on three foundational principles:


1. **Adaptive Retrieval Strategy Selection**
  * Instead of using a single fixed retrieval approach (e.g., FAISS, BM25, or hybrid retrieval), Adaptive-RAG:

  * Dynamically chooses between sparse (BM25, TF-IDF), dense (FAISS, OpenAI Embeddings), or hybrid retrieval.
  * Uses query classification to identify optimal retrieval methods (e.g., factual, exploratory, opinion-based).
  * Adjusts retrieval depth (k) based on the query's complexity.

2. **Context-Aware Query Expansion**

  * Uses query rewriting techniques with LLMs to expand, refine, or simplify the query.
  * Helps in cases where user queries are ambiguous or too broad.
  * Uses self-querying retrievers to reformat the query before retrieval.

3. **Dynamic Retrieval Weighting**

  * Assigns dynamic weights to retrieval results based on:
    * Confidence scores (how well documents match the query).
    * Semantic similarity (using cosine similarity on embeddings).
    * Keyword overlap (BM25 score-based filtering).
    
By balancing retrieval results, Adaptive-RAG ensures more diverse and relevant responses.

# **Core**
**Adaptive-RAG dynamically selects the best retrieval strategy for each query:**

1. **Query Classification:** Determines if a query needs retrieval (and which type)

2. **Retrieval Options:**
  * **Dense Retrieval:** For semantic/similarity searches (e.g., "Explain climate change")

  * **Sparse Retrieval:** For keyword-based searches (e.g., "GDP of France in 2023")

  * **Hybrid:** Combines both methods for complex queries

3. **Fallback Mechanism:** Uses the LLM's internal knowledge if no relevant documents are found

# **Implementation**

In [1]:
!pip install -qU langchain langchain-openai langchain-community faiss-cpu rank_bm25

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.7/412.7 kB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 3.7 MB/s eta 0:00:00


In [2]:
import openai
from google.colab import userdata
import os


openai_api= userdata.get("OPENAI_API_KEY")

In [3]:
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [4]:
# 1. Initialize components
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0,openai_api_key=openai_api)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",openai_api_key=openai_api)

In [5]:
# Sample documents
docs = [
    "The French Revolution began in 1789.",
    "Napoleon became Emperor in 1804.",
    "World War II ended in 1945."
]

In [6]:
# 2. Create retrievers
# Sparse (keyword-based)
bm25_retriever = BM25Retriever.from_texts(docs)
bm25_retriever.k = 2

# Dense (semantic)
vectorstore = FAISS.from_texts(docs, embeddings)
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

In [7]:
# 3. Query classifier
def classify_query(query: str) -> str:
    prompt = """Classify the query type:
    - 'simple' (needs no retrieval, e.g., "Hello!")
    - 'fact' (needs keyword search, e.g., "When did WWII end?")
    - 'complex' (needs semantic search, e.g., "Explain the French Revolution")

    Query: {query}"""

    response = llm.invoke(prompt.format(query=query)).content.lower()
    return "complex" if "complex" in response else "fact" if "fact" in response else "simple"

In [8]:
# 4. Adaptive retriever
def adaptive_retrieve(query: str):
    query_type = classify_query(query)

    if query_type == "simple":
        return []  # No retrieval needed
    elif query_type == "fact":
        return bm25_retriever.invoke(query)
    else:
        return faiss_retriever.invoke(query)

In [9]:
# 5. QA chain
template = """Answer using ONLY these documents:
{context}

Question: {query}
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnablePassthrough.assign(
        context=lambda x: adaptive_retrieve(x["query"])
    )
    | prompt
    | llm
    | StrOutputParser()
)

In [10]:
# 6. Test
queries = [
    "Hello! How are you?",          # Simple (no retrieval)
    "When did WWII end?",           # Fact (BM25)
    "Explain the French Revolution" # Complex (FAISS)
]

for query in queries:
    print(f"Query: {query}")
    print("Answer:", chain.invoke({"query": query}))
    print("\n" + "-"*50 + "\n")

Query: Hello! How are you?
Answer: I am doing well, thank you for asking.

--------------------------------------------------

Query: When did WWII end?
Answer:  WWII ended in 1945.

--------------------------------------------------

Query: Explain the French Revolution
Answer: The French Revolution began in 1789. Napoleon became Emperor in 1804.

--------------------------------------------------



## **Key Components**
1. **Query Classifier:**


```
def classify_query(query: str) -> str:
    # Uses LLM to categorize queries
```
2. **Retrieval Router:**


```
def adaptive_retrieve(query: str):
    # Selects retrieval method based on query type
```
3. **Fallback Mechanism:**


```
template = """Answer using ONLY these documents:
{context}  # Empty list for simple queries
```





